# Model Calibration as Part of Model Evaluation in a Model Evolution System
Model calibration serves the purpose of comparing a (data) model revision against a benchmark (expert) model. It is a validation process that ensures external quality assurance. The key question asked during the calibration process is: *"Does the proposed new model meet stakeholder needs?"*

The calibration process captures the following 3 questions that determine the extend to which a proposed model is fit for purpose:
1. __Data integrity__: How good is my data?
2. __Model integrity__: How well does my new model match my existing model?
3. __Abstraction integrity__: How well does my model represent reality?

## Generate Data Model

### Model Setup

In [1]:
import features.ts as ts
import evaluation.evalhelpers as eh
import evaluation.calibration as ec

In [2]:
year = 2011
experiment_dir = 'exp1' # sub-directory in which inferred customer classes are saved

# Get the demand profile (kVA) for all profile IDs for one year.
pp = ts.getProfilePower(year)

### Time Series Selection and Feature Extraction

In [3]:
# Aggregate demand profiles by month
aggpp = ts.aggProfilePower(pp, 'M')

# Generate the maximum demand for each profile ID
md = ts.maxDemand(pp)

# Calculate the annualised monthly power demand for each profile ID
amd = ts.annualIntervalDemand(aggpp)

# Calculate the power demand by hour, day type and month for each profile ID
adtd = ts.aggDaytypeDemand(pp)

### Construct Class Model

In [11]:
%%capture
# Generate demand summary
ods = eh.observedDemandSummary(amd, year, experiment_dir)

# Generate hourly demand profiles for each day type and month
ohp = eh.observedHourlyProfiles(adtd, year, experiment_dir)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

## Obtain Uncertainty Index as an Indication of Data Integrity
The purpose of an uncertainty index is to assess the data integrity. The key questions that the uncertainty index answers are:
1. *Do I have enough representative data?*
2. *Is the data sufficiently reliable to construct a model with integrity?*

The uncertainty index is calculated by establishing whether the sample size is sufficient to draw a conclusion about a certain characteristic feature of the model. In this system it is derived from the:
* number of profiles observed
* number of valid observations per observed profile

Moreover, for a model to be valid, it must share the same baseline as the benchmark model (eg same year, same region).

In [8]:
ec.uncertaintystats(ods)[:8]

AnswerID_count  valid_obs_ratio
customer_class      index                                 
informal_settlement count       15.000000        15.000000
                    mean        13.266667         0.910585
                    std         19.436588         0.089651
                    min          2.000000         0.705574
                    25%          3.000000         0.893042
                    50%          5.000000         0.949663
                    75%         15.000000         0.970046
                    max         78.000000         0.976135

In [9]:
ec.uncertaintystats(ods)[:8]

AnswerID_count  valid_obs_ratio
customer_class      index                                 
informal_settlement count       15.000000        15.000000
                    mean        13.266667         0.910585
                    std         19.436588         0.089651
                    min          2.000000         0.705574
                    25%          3.000000         0.893042
                    50%          5.000000         0.949663
                    75%         15.000000         0.970046
                    max         78.000000         0.976135